In [ ]:
import numpy as np
from rasterio.features import rasterize, geometry_mask
from rasterio.transform import Affine
import shapely
from shapely.geometry import Polygon
import rasterio
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook # enables plot interface in J notebook
# init bokeh
output_notebook()
from typing import Tuple, List, Union


In [ ]:
image_width=200
image_height=100

# y mettre l'origine de la ROI - en effet les polygones sont definis en coordonnees image sur le modele entier.
# l'origine correspond donc a l'origine de la grille en coordonnees image
origin_x = 0.5
origin_y = 0.5

# s'apparente a la resolution de la grille - si appliquer sur une grille pleine resolution y mettre 1.
# si la grille est surechantillonnee y mettre l'inverse du surech ; ex surech 4 => 0.25
# si la grille est sous-echantillonner y mettre le facteur de sous-ech : sous-ech de 4 => 4
pixel_size_x = 1
pixel_size_y = 1

polygones = [(Polygon([(10.5, 10.5), (20.5, 10.5), (23.5, 7.5), (27.5, 20.5), (10.5, 20.5), (10.5, 10.5)]), 0), 
             (Polygon([(150.5, 50.5), (160.5, 50.5), (160.5, 60.5), (150.5, 60.5), (150.5, 50.5)]), 0)]

transform = Affine(pixel_size_x, 0, origin_x, 0, pixel_size_y, origin_y)

# Geometries sous la forme d'une liste de polygon
geometries = [(polygon, value) for polygon, value in polygones]
# Geometries sous la forme d'un polygone
geometries = shapely.unary_union([p for p,_ in polygones])
if geometries.geom_type == 'MultiPolygon':
    geometries = [(geom, 0) for geom in geometries.geoms]
#display(geometries.wkt)

mask = rasterize(
    geometries,
    out_shape=(image_height, image_width),
    transform=transform,
    fill=1,
    dtype='uint16')

plt.imshow(mask, cmap='viridis')
plt.colorbar(label='valeurs')
plt.show()

In [ ]:
def rasterize_geometry_on_regular_grid(
        grid_shape: Tuple[float, float],
        grid_origin: Tuple[float, float],
        grid_resolution: Tuple[float, float],
        geometry: Union[shapely.geometry.Polygon, shapely.geometry.MultiPolygon],
        reduce: bool = False,
        ) -> Union[np.ndarray, int]:
    """Rasterize a geometry on a grid
    
    The grid_resolution parameter is used to set the ratio between the geometry
    coordinates and the grid resolution. It corresponds to the grid pixel size
    in regards to the geometry definition.
    
    Args:
        grid_shape: the grid size given as a tuple (number of rows, number of
            columns).
        grid_origin: the coordinates (row, col) of the grid first element (0,0)
            in the image coordinates reference system used by the geometry.
        grid_resolution: the grid relative pixel size towards the geometry 
            coordinate reference system.
        geometry: the unmasked geometry definition. It can either be a single
            polygon or a multi_polygon.
            The geometry must be defined in the same reference frame as that 
            intrinsic to the image targeted by the grid, considering the scale
            factor (oversampling/undersampling) and with a possible shift of its
            origin.
            The coordinates are here supposed to be given following the standard
            (x: col, y: row) order.
        reduce: a boolean option that returns the corresponding scalar (0 or 1)
            if the resulting raster if fully filled with that scalar
    
    Returns:
        the corresponding raster or the scalar if the raster is filled with it
        and reduce is set to True.
    
    """
    transform = Affine(grid_resolution[1], 0, grid_origin[1],
              0, grid_resolution[0], grid_origin[0])
    
    def geometry_to_valued_polygon_list(geom: shapely.geometry
            ) -> List[Polygon]:
        """Convert a geometry supposed to be a MultiPolygon or a Polygon to a
        list of polygons.
        
        Args:
            geom: the geometry to convert
        
        Returns:
            a list of polygons or an empty list if geometry's type mismatch.
        """
        geom_list = []
        if geom.geom_type == 'MultiPolygon':
            #geom_list = [(poly, 0) for poly in geom.geoms]
            geom_list = [poly for poly in geom.geoms]
        elif geom.geom_type == 'Polygon':
            geom_list = [geom]
            #geom_list = [(geom, 0)]
        return geom_list
    
    polygons = []
    try:
        polygons = geometry_to_valued_polygon_list(geometry)
    except AttributeError:
        # We suppose here geometries are passed as a list of geometries
        for geom in geometry:
            polygons.extend(geometry_to_valued_polygon_list(geom))

    if len(polygons) > 0:
        
        raster = geometry_mask(
                polygons,
                out_shape=(grid_shape[0], grid_shape[1]),
                transform=transform,
                all_touched=False,
                )
                #fill=1,
                #dtype='uint8')

        if reduce:
            if np.all(raster==0):
                raster = 0
            elif np.all(raster!=0):
                raster = 1
    
    else:
        if reduce:
            raster = 0
        else:
            raster = np.zeros(grid_shape, dtype=np.uint8)
    
    return raster

In [ ]:
from bokeh.models import ColumnDataSource
import shapely
# grid x, y
#mask = np.zeros(shape)
#mask[0,0]=1
#mask[2, 5]=1

shape=(15, 12)
origin=(0.5, 0.5)
resolution=(1, 1.0)
geometry = []
epsilon=0.
gx, gy = 1.5, 2.5
gw, gh = 4, 10
#geometry = [Polygon([(1.5, 3.5), (2.5, 3.5), (2.5, 5.5), (1.5, 5.5)])]
geometry = [Polygon([(gx-epsilon, gy-epsilon),
                     (gx+gw+epsilon, gy-epsilon),
                     (gx+gw+epsilon, gy+gh+epsilon),
                     (gx-epsilon, gy+gh+epsilon),
                     (gx-epsilon, gy-epsilon)])]
#geometry = [Polygon([(1.5,3.5), (4.5, 5.5), (8.5, 10.5), (1.5, 9.5), (1.5, 3.5)])]
mask = rasterize_geometry_on_regular_grid(shape, origin, resolution, geometry, reduce=False)

#display(mask)
#display(mask.dtype)
display(mask.shape)

# Create grid coordinates
i = np.arange(shape[1])
j = np.arange(shape[0])
ii, jj = np.meshgrid(i,j,indexing='xy')
x = np.linspace(origin[1], origin[1]+(shape[1]-1)*resolution[1], shape[1])
y = np.linspace(origin[0], origin[0]+(shape[0]-1)*resolution[0], shape[0])
pixels_centers_x, pixels_centers_y = np.meshgrid(x,y,indexing='xy')

# Naiv mask computation
mask_bis = np.ones(shape, dtype=np.int8)
for i,j in zip(ii.flatten(), jj.flatten()):
    p = shapely.geometry.Point(pixels_centers_x[j,i], pixels_centers_y[j,i])
    for geom in geometry:
        #print(geom, p)
        if geom.intersects(p):
        #if geom.contains(p):
            #print(geom.distance(p)) # could also use distance if faster => 0 if point is in
            mask_bis[j,i] = 0
        

plot_res = 40
p = figure(width=shape[1]*plot_res, height=shape[0]*plot_res)
p.x_range.range_padding = p.y_range.range_padding = 0.5
p.y_range.flipped = True

p.image(image=[mask], x=pixels_centers_x[0,0]-resolution[1]/2 , y=pixels_centers_y[0,0]-resolution[0]/2, dw=shape[1]*resolution[1], dh=shape[0]*resolution[0], alpha=0.4, palette="Viridis256")
p.image(image=[mask_bis], x=pixels_centers_x[0,0]-resolution[1]/2 , y=pixels_centers_y[0,0]-resolution[0]/2, dw=shape[1]*resolution[1], dh=shape[0]*resolution[0], alpha=0.4, palette="Greys256")
# plot grid centers
p.cross(pixels_centers_x.flatten(), pixels_centers_y.flatten(), size=2, color="black", alpha=0.8)

geom_x, geom_y = [], []
[(geom_x.append(list(polygon.exterior.coords.xy[0])), geom_y.append(list(polygon.exterior.coords.xy[1]))) for polygon in geometry ]
p.patches('x', 'y', source = ColumnDataSource(dict(x = geom_x, y = geom_y)), line_color = "red", line_width = 0.5, fill_color=None)


p.grid.grid_line_width = 0.5
show(p)


from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))